# ETL Report for Census Annual Business Survey
## Introduction
Every year The United States Census Bureau conducts their [Annual Business Survey (ABS)](https://www.census.gov/programs-surveys/abs/about.html) electronically. It provides information on selected economic and demographic characteristics for businesses and business owners by sex, ethnicity, race, and veteran status.

In this project, we use the ABS to learn more about demographics in The United States' businesses.
### Data Sources
All of our data was sourced from The United States Census Bureau [2019 Annual Business Survey](https://www.census.gov/data/developers/data-sets/abs.2019.html) by calling their APIs:
* [Company Summary](https://api.census.gov/data/2018/abscs.html)
* [Characteristics of Businesses](https://api.census.gov/data/2018/abscb.html)
* [Characteristics of Business Owners](https://api.census.gov/data/2018/abscbo.html)
* [Technology Characteristics of Businesses](https://api.census.gov/data/2018/abstcb.html)

Below you will find an outline for the Extract, Transform, and Load (ETL) process for the data.

## Extract
1. Import the `api_key` from the config file in order to run API requests
2. Set up API requests with appropriate links, formatting the `api_key`
3. Call the API for each request and store the outcome as a DataFrame using `API2DataFrame`

In [1]:
#import relevant libraries
import requests
from config import api_key
import json
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
#URLs
business_owner_url = f'https://api.census.gov/data/2018/abscbo?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F,OWNPDEMP_S,OWNPDEMP_S_F,OWNPDEMP_PCT_S,OWNPDEMP_PCT_S_F&for=us:*&for=QDESC_LABEL=ACQBUS&key={api_key}'

business_characteristics_url = f'https://api.census.gov/data/2018/abscb?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,PAYANN_PCT_S_F&for=us:*&for=QDESC_LABEL=SPOUSES&key={api_key}'

company_summary_url = f'https://api.census.gov/data/2018/abscs?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F&for=us:*&key={api_key}'

tech_characteristics_url = f'https://api.census.gov/data/2018/abstcb?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,NSFSZFI,NSFSZFI_LABEL,FACTORS_P,FACTORS_P_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,PAYANN_PCT_S_F&for=us:*&key={api_key}'

In [3]:
def API2DataFrame(url):
    '''
    url: str
    
    takes in url for API and returns a pandas DataFrame
    '''
    #Gather data from the API
    response = requests.get(url)
    #turn into json
    response = response.json()
    #Load data into DataFrame
    df = pd.DataFrame(data = response[1:], columns = response[0])
    
    return df

Let's begin by calling from the API and loading the data into a `pandas` DataFrame

In [4]:
#DataFrames
business_owner_characteristics_raw = API2DataFrame(business_owner_url)

In [ ]:

business_characteristics_raw = API2DataFrame(business_characteristics_url)

In [4]:

company_summary_raw = API2DataFrame(company_summary_url)

In [ ]:
tech_characteristics_raw = API2DataFrame(rech_characteristics_url)

## Transform
1. Remove unwanted data columns from the DataFrame (different for each)
2. Drop columns and rows where all entries are NA
3. Remove records where there is a 'withheld' flag (see D flag in [ABS Documentation](https://www2.census.gov/programs-surveys/abs/technical-documentation/api/ABS_API_CBO-1-26-2021.pdf))
4. After filtering flags appropriately, drop flag columns

These steps can all be done by calling `cleanDataFrame()` and feeding it:
* `dataframe` : a `pandas` DataFrame (e.g. those pre-loaded in the Extract step above)
* `drop_columns` : a list of columns to be dropped from the DataFrame
* `filter_columns` : a list of columns to check for flags and filter
* `rename_cols` : a dictionary of the columns to be renamed along with their new name

The function will then return a cleaned DataFrame

In [5]:
def cleanDataFrame(dataframe, drop_columns = [], filter_columns = [], rename_cols = {}):
    '''
    Takes in a pandas dataframe and drops unwanted columns and cleans data according to flag/filters
    
    Parameters
    ----------
    dataframe: a pandas DataFrame
    
    drop_columns: list of columns to be dropped from dataframe
    
    filter_columns: list of columns that act as filter/flags for the record
    
    Returns
    --------
    dataframe : the cleaned dataset as a pandas dataframe
    
    '''
    
    #Remove unwanted columns
    dataframe = dataframe.drop(columns = drop_columns)
    
    #Remove all columns and rows that have all null values
    dataframe.dropna(axis='columns', how='all',inplace=True)
    dataframe.dropna(axis='rows', how='all',inplace=True)
    
    #Remove all records where there is a D flag
    for column in filter_columns:
        #Remove records for one flag at a time
        dataframe.drop(dataframe.loc[dataframe[column] == 'D'].index, inplace=True)
        
    #After filtering flags, we can remove flag columns
    dataframe.drop(columns = filter_columns, inplace = True)
    
    #Rename the columns to something more intutitive
    dataframe.rename(columns = rename_cols, inplace=True)
    
    #Change numeric columns to int or float
    for col in dataframe.columns:
        if ('Percentage' or 'Err') in col:
            dataframe[col] = dataframe[col].astype(float)
        elif 'Number' in col:
            dataframe[col] = dataframe[col].astype(int)
    
    
    
    return dataframe
    

## Business Owner Characteristics

In [6]:
business_owner_characteristics_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104622 entries, 0 to 104621
Data columns (total 26 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   GEO_ID            104622 non-null  object
 1   NAME              104622 non-null  object
 2   NAICS2017         104622 non-null  object
 3   NAICS2017_LABEL   104622 non-null  object
 4   OWNER_SEX         104622 non-null  object
 5   OWNER_SEX_LABEL   104622 non-null  object
 6   OWNER_ETH         104622 non-null  object
 7   OWNER_ETH_LABEL   104622 non-null  object
 8   OWNER_RACE        104622 non-null  object
 9   OWNER_RACE_LABEL  104622 non-null  object
 10  OWNER_VET         104622 non-null  object
 11  OWNER_VET_LABEL   104622 non-null  object
 12  QDESC             104622 non-null  object
 13  QDESC_LABEL       104622 non-null  object
 14  OWNCHAR           104622 non-null  object
 15  OWNCHAR_LABEL     104622 non-null  object
 16  YEAR              104622 non-null  obj

You may notice that some of these columns have codes and are repeated with labels. Let's go ahead and remove the columns that do not add to the data/are redundant

In [7]:
# For each data frame, we'll want to drop specific columns
# Business owner characteristics: GEO_ID, NAICS2017, OWNER_SEX, OWNER_ETH, OWNER_RACE, OWNER_VET, QDESC, OWNCHAR, YEAR, state
drop_columns = ['GEO_ID', 'NAICS2017', 'OWNER_SEX', 'OWNER_ETH', 'OWNER_RACE',
                'OWNER_VET', 'QDESC', 'OWNCHAR', 'YEAR', 'us']

flag_columns = ['OWNPDEMP_F', 'OWNPDEMP_PCT_F', 'OWNPDEMP_S_F', 'OWNPDEMP_PCT_S_F']

renaming = {
    'NAME' : 'Region',
    'NAICS2017_LABEL' : 'IndustrySector',
    'OWNER_SEX_LABEL' : 'OwnerSex',
    'OWNER_ETH_LABEL' : 'OwnerEthnicity',
    'OWNER_RACE_LABEL' : 'OwnerRace',
    'OWNER_VET_LABEL' : 'VeteranStatus',
    'QDESC_LABEL' : 'QuestionDescription',
    'OWNCHAR_LABEL' : 'OwnerResp',
    'OWNPDEMP' : 'NumberOfOwners',
    'OWNPDEMP_PCT' : 'PercentageOfOwners',
    'OWNPDEMP_S' : 'NumOfOwnersErr',
    'OWNPDEMP_PCT_S' : 'PercentageOfOwnersErr'
}

business_owner_characteristics = cleanDataFrame(
    business_owner_characteristics_raw, drop_columns = drop_columns,
    filter_columns = flag_columns, rename_cols = renaming
)

In [9]:
business_owner_characteristics.to_csv(r'Data\business_owner_characteristics.csv')

In [10]:
business_owner_characteristics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92564 entries, 0 to 104621
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Region                 92564 non-null  object 
 1   Sector                 92564 non-null  object 
 2   OwnerSex               92564 non-null  object 
 3   OwnerEthnicity         92564 non-null  object 
 4   OwnerRace              92564 non-null  object 
 5   VeteranStatus          92564 non-null  object 
 6   QuestionDescription    92564 non-null  object 
 7   OwnerResp              92564 non-null  object 
 8   NumberOfOwners         92564 non-null  int32  
 9   PercentageOfOwners     92564 non-null  float64
 10  NumOfOwnersErr         92564 non-null  object 
 11  PercentageOfOwnersErr  92564 non-null  float64
dtypes: float64(2), int32(1), object(9)
memory usage: 8.8+ MB


In [27]:
business_owner_characteristics['NAICS2017_LABEL'].value_counts()

Total for all sectors                                                       5725
Professional, scientific, and technical services                            5483
Manufacturing                                                               5169
Health care and social assistance                                           5046
Wholesale trade                                                             4990
Retail trade                                                                4912
Construction                                                                4901
Accommodation and food services                                             4887
Administrative and support and waste management and remediation services    4863
Other services (except public administration)                               4770
Transportation and warehousing                                              4749
Information                                                                 4685
Real estate and rental and l

## Business Characteristics

In [30]:
business_characteristics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48737 entries, 1 to 48737
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   GEO_ID             48737 non-null  object
 1   NAME               48737 non-null  object
 2   NAICS2017          48737 non-null  object
 3   NAICS2017_LABEL    48737 non-null  object
 4   SEX                48737 non-null  object
 5   SEX_LABEL          48737 non-null  object
 6   ETH_GROUP          48737 non-null  object
 7   ETH_GROUP_LABEL    48737 non-null  object
 8   RACE_GROUP         48737 non-null  object
 9   RACE_GROUP_LABEL   48737 non-null  object
 10  VET_GROUP          48737 non-null  object
 11  VET_GROUP_LABEL    48737 non-null  object
 12  QDESC              48737 non-null  object
 13  QDESC_LABEL        48737 non-null  object
 14  BUSCHAR            48737 non-null  object
 15  BUSCHAR_LABEL      48737 non-null  object
 16  YEAR               48737 non-null  objec

You may notice that some of these columns have codes and are repeated with labels. Let's go ahead and remove the columns that do not add to the data/are redundant

In [31]:
# For each data frame, we'll want to drop specific columns
# Business characteristics: GEO_ID, NAICS2017, SEX, ETH_GROUP, RACE_GROUP, VET_GROUP, QDESC, BUSCHAR, YEAR, us
drop_columns = ['GEO_ID', 'NAICS2017', 'SEX', 'ETH_GROUP', 'RACE_GROUP', 'VET_GROUP', 'QDESC', 'BUSCHAR', 'YEAR', 'us']
flag_columns = ['FIRMPDEMP_F', 'FIRMPDEMP_PCT_F', 'RCPPDEMP_F', 'RCPPDEMP_PCT_F', 'EMP_F', 'EMP_PCT_F', 'PAYANN_F',
                'PAYANN_PCT_F', 'FIRMPDEMP_S_F', 'FIRMPDEMP_PCT_S_F', 'RCPPDEMP_S_F', 'RCPPDEMP_PCT_S_F', 'EMP_S_F',
                'EMP_PCT_S_F', 'PAYANN_S_F', 'PAYANN_PCT_S_F']

business_characteristics = cleanDataFrame(business_characteristics_raw, drop_columns = drop_columns, filter_columns = flag_columns)

In [32]:
business_characteristics

,NAME,NAICS2017_LABEL,SEX_LABEL,ETH_GROUP_LABEL,RACE_GROUP_LABEL,VET_GROUP_LABEL,QDESC_LABEL,BUSCHAR_LABEL,FIRMPDEMP,FIRMPDEMP_PCT,...,PAYANN,PAYANN_PCT,FIRMPDEMP_S,FIRMPDEMP_PCT_S,RCPPDEMP_S,RCPPDEMP_PCT_S,EMP_S,EMP_PCT_S,PAYANN_S,PAYANN_PCT_S
1,United States,Total for all sectors,Total,Total,Total,Total,OWNRNUM,All firms,5722142,0.0,...,6937240308,0.0,0.0,0.0,1.2,0.0,0.5,0.0,0.5,0.0
2,United States,Total for all sectors,Total,Total,Total,Total,OWNRNUM,1 person,1628501,59.1,...,454218390,14.2,0.4,0.6,1.3,66.8,1.1,61.6,1.3,67.2
3,United States,Total for all sectors,Total,Total,Total,Total,OWNRNUM,2 to 4 people,1024784,37.2,...,478308387,15.0,0.6,0.3,3.0,87.0,1.5,68.9,2.7,77.9
4,United States,Total for all sectors,Total,Total,Total,Total,OWNRNUM,5 to 10 people,49436,1.8,...,75150602,2.4,2.5,0.1,7.8,12.4,4.8,7.0,4.2,10.1
5,United States,Total for all sectors,Total,Total,Total,Total,OWNRNUM,11 or more people,31846,1.2,...,986906070,30.9,2.7,0.3,0.9,62.3,0.8,49.4,0.9,53.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48732,United States,Industries not classified,Equally male/female,Equally Hispanic/non-Hispanic,Total,Total,SPOUSES,Total reporting,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48734,United States,Industries not classified,Total,Non-Hispanic,Total,Total,SPOUSES,All firms,20665,0.0,...,1147897,0.0,0.4,0.0,0.0,0.0,20.8,0.0,3.1,0.0
48735,United States,Industries not classified,Total,Non-Hispanic,Total,Total,SPOUSES,Jointly owned and equally operated by spouses,1587,28.4,...,69723,31.8,9.5,2.5,0.0,0.0,51.1,19.3,12.6,2.9
48736,United States,Industries not classified,Total,Non-Hispanic,Total,Total,SPOUSES,Jointly owned but primarily operated by male s...,1287,23.1,...,37239,17.0,8.4,1.8,0.0,0.0,36.7,9.8,13.1,2.1


## Company Summary

In [34]:
company_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19458 entries, 1 to 19458
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   GEO_ID            19458 non-null  object
 1   NAME              19458 non-null  object
 2   NAICS2017         19458 non-null  object
 3   NAICS2017_LABEL   19458 non-null  object
 4   SEX               19458 non-null  object
 5   SEX_LABEL         19458 non-null  object
 6   ETH_GROUP         19458 non-null  object
 7   ETH_GROUP_LABEL   19458 non-null  object
 8   RACE_GROUP        19458 non-null  object
 9   RACE_GROUP_LABEL  19458 non-null  object
 10  VET_GROUP         19458 non-null  object
 11  VET_GROUP_LABEL   19458 non-null  object
 12  EMPSZFI           19458 non-null  object
 13  EMPSZFI_LABEL     19458 non-null  object
 14  YEAR              19458 non-null  object
 15  FIRMPDEMP         19458 non-null  object
 16  FIRMPDEMP_F       7516 non-null   object
 17  RCPPDEMP    

In [35]:
# For each data frame, we'll want to drop specific columns
# company summary: GEO_ID, NAICS2017, SEX, ETH_GROUP, RACE_GROUP, VET_GROUP, EMPSZFI, YEAR, us
drop_columns = ['GEO_ID', 'NAICS2017', 'SEX', 'ETH_GROUP', 'RACE_GROUP', 'VET_GROUP', 'EMPSZFI', 'YEAR', 'us']
flag_columns = ['FIRMPDEMP_F', 'RCPPDEMP_F', 'EMP_F', 'PAYANN_F', 'FIRMPDEMP_S_F', 'RCPPDEMP_S_F', 'EMP_S_F', 'PAYANN_S_F']

company_summary = cleanDataFrame(company_summary_raw, drop_columns = drop_columns, filter_columns = flag_columns)

In [36]:
company_summary

,NAME,NAICS2017_LABEL,SEX_LABEL,ETH_GROUP_LABEL,RACE_GROUP_LABEL,VET_GROUP_LABEL,EMPSZFI_LABEL,FIRMPDEMP,RCPPDEMP,EMP,PAYANN,FIRMPDEMP_S,RCPPDEMP_S,EMP_S,PAYANN_S
1,United States,Total for all sectors,Female,Hispanic,Total,Total,All firms,84496,91103725,722654,21975675,3.0,8.9,7.7,7.0
2,United States,Total for all sectors,Female,Hispanic,Total,Total,Firms with no employees,10036,3233978,0,499996,4.7,19.2,0.0,13.5
3,United States,Total for all sectors,Female,Hispanic,Total,Total,Firms with 1 to 4 employees,47105,15792524,92631,2808773,4.7,8.2,4.8,7.4
4,United States,Total for all sectors,Female,Hispanic,Total,Total,Firms with 5 to 9 employees,12523,10688294,83449,2601817,5.9,4.3,5.6,6.3
5,United States,Total for all sectors,Female,Hispanic,Total,Total,Firms with 10 to 19 employees,8516,14105603,116479,3506384,6.0,16.2,6.7,10.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19452,United States,"Mining, quarrying, and oil and gas extraction",Equally male/female,Total,Nonminority,Total,Firms with 5 to 9 employees,451,0,3101,178609,37.9,0.0,38.8,37.4
19453,United States,"Mining, quarrying, and oil and gas extraction",Equally male/female,Total,Nonminority,Total,Firms with 10 to 19 employees,234,0,3313,168872,43.8,0.0,46.3,42.1
19454,United States,"Mining, quarrying, and oil and gas extraction",Equally male/female,Total,Nonminority,Total,Firms with 20 to 49 employees,0,0,0,0,0.0,0.0,0.0,0.0
19455,United States,"Mining, quarrying, and oil and gas extraction",Equally male/female,Total,Nonminority,Total,Firms with 50 to 99 employees,25,0,1369,108597,27.8,0.0,20.0,16.7


In [37]:
company_summary['NAICS2017_LABEL'].value_counts()

Total for all sectors                                                       1133
Professional, scientific, and technical services                             996
Health care and social assistance                                            963
Accommodation and food services                                              956
Administrative and support and waste management and remediation services     956
Manufacturing                                                                955
Transportation and warehousing                                               916
Retail trade                                                                 901
Construction                                                                 895
Wholesale trade                                                              860
Information                                                                  834
Other services (except public administration)                                820
Finance and insurance       